## Train a  Bernoulli-Bernoulli RBM (BBRBM) to fit XOR calculations

In [1]:
import numpy as np
import timeit
from tfrbm.bbrbm import BBRBM

In [2]:
bm = BBRBM(n_visible=3,n_hidden=13)

### dataset[i][0] xor dataset[i][1] = dataset[i][2]

In [3]:
dataset = np.array([[0,0,0],[0,1,1],[1,0,1],[1,1,0]])

### Train and predict same data in same epoch

In [4]:
for i in range(4):
    x = dataset[i:i+1].copy()
    x2 = dataset[i:i+1].copy()
    x2[0,2] = 1-x2[0,2]
    print "x:",x
    print "x2:",x2
    print "Training model with x"
    err = bm.fit(x,n_epoches=1000)
    print "model reconstructed x:",np.round(bm.reconstruct(x))
    print "model reconstructed x2:",np.round(bm.reconstruct(x2))
    print "Training done"

    positions_to_predict = [2]
    prediction = bm.predict(x,positions_to_predict=positions_to_predict) # best result, energy for all possibilities
    print "model predicting x third digit to be:",prediction[0].tolist()
    prediction = bm.predict(x2,positions_to_predict=positions_to_predict)
    print "model predicting x2 third digit to be:",prediction[0].tolist()
    print "------------------------------------"

x: [[0 0 0]]
x2: [[0 0 1]]
Training model with x
model reconstructed x: [[0. 0. 0.]]
model reconstructed x2: [[0. 0. 0.]]
Training done
model predicting x third digit to be: [0]
model predicting x2 third digit to be: [0]
------------------------------------
x: [[0 1 1]]
x2: [[0 1 0]]
Training model with x
model reconstructed x: [[0. 1. 1.]]
model reconstructed x2: [[0. 1. 1.]]
Training done
model predicting x third digit to be: [1]
model predicting x2 third digit to be: [1]
------------------------------------
x: [[1 0 1]]
x2: [[1 0 0]]
Training model with x
model reconstructed x: [[1. 0. 1.]]
model reconstructed x2: [[1. 0. 1.]]
Training done
model predicting x third digit to be: [1]
model predicting x2 third digit to be: [1]
------------------------------------
x: [[1 1 0]]
x2: [[1 1 1]]
Training model with x
model reconstructed x: [[1. 1. 0.]]
model reconstructed x2: [[1. 1. 0.]]
Training done
model predicting x third digit to be: [0]
model predicting x2 third digit to be: [0]
-----

In [5]:
for i in range(4):
    x = dataset[i:i+1].copy()
    x2 = dataset[i:i+1].copy()
    x2[0,2] = 1-x2[0,2]
    print "x:",x
    print "x2:",x2
    
    positions_to_predict = [2]
    print "model reconstructed x:",np.round(bm.reconstruct(x))
    print "model reconstructed x2:",np.round(bm.reconstruct(x2))
    print "------------------------------------"

x: [[0 0 0]]
x2: [[0 0 1]]
model reconstructed x: [[1. 1. 0.]]
model reconstructed x2: [[1. 1. 0.]]
------------------------------------
x: [[0 1 1]]
x2: [[0 1 0]]
model reconstructed x: [[1. 1. 0.]]
model reconstructed x2: [[1. 1. 0.]]
------------------------------------
x: [[1 0 1]]
x2: [[1 0 0]]
model reconstructed x: [[1. 1. 0.]]
model reconstructed x2: [[1. 1. 0.]]
------------------------------------
x: [[1 1 0]]
x2: [[1 1 1]]
model reconstructed x: [[1. 1. 0.]]
model reconstructed x2: [[1. 1. 0.]]
------------------------------------


In [6]:
testset = np.array([[0,0,0],[0,0,1]])
testset = np.repeat(testset, [8,2],axis=0)
print testset
start = timeit.default_timer()
err = bm.fit(testset,n_epoches=1000)
stop = timeit.default_timer()
print('Time: ', stop - start) 
print "model reconstructed x:",np.round(bm.reconstruct(testset[0:1]))
print "model reconstructed x:",np.round(bm.reconstruct(testset[-2:-1]))

[[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 1]
 [0 0 1]]
('Time: ', 0.7905220985412598)
model reconstructed x: [[0. 0. 0.]]
model reconstructed x: [[0. 0. 0.]]


In [7]:
testset = np.array([[0,0,0],[0,0,1]])
testset = np.repeat(testset, [2,8],axis=0)
print testset
start = timeit.default_timer()
err = bm.fit(testset,n_epoches=1000)
stop = timeit.default_timer()
print('Time: ', stop - start) 
print "model reconstructed x:",np.round(bm.reconstruct(testset[0:1]))
print "model reconstructed x:",np.round(bm.reconstruct(testset[-2:-1]))

[[0 0 0]
 [0 0 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]]
('Time: ', 0.7799370288848877)
model reconstructed x: [[0. 0. 0.]]
model reconstructed x: [[0. 0. 1.]]


In [8]:
bm = BBRBM(n_visible=3,n_hidden=13)
testset = np.array([[0,0,0],[0,0,1]])
testset = np.repeat(testset, [2,8],axis=0)
# print testset
start = timeit.default_timer()
err = bm.fit(testset,n_epoches=1000)
stop = timeit.default_timer()
print('Time: ', stop - start) 
print "model reconstructed x:",np.round(bm.reconstruct(testset[0:1]))
print "model reconstructed x:",np.round(bm.reconstruct(testset[-2:-1]))

('Time: ', 0.986940860748291)
model reconstructed x: [[0. 0. 1.]]
model reconstructed x: [[0. 0. 1.]]
